# Korišćenje i podešavanje algoritama mašinskog učenja za rešavanje problema klasifikacije

Izbor algoritma mašinskog učenja ima veliki uticaj na tačnost predviđanja modela. On zavisi od velikog broja faktora kao što su broj podataka, raspodela podataka različitih veličina, itd. Osnovna podela algoritama je na parametarske i neparametarske. 

Parametarski algoritmi podrazumevaju linearnu funkciju mapiranja, samim tim i znatno jednostavnije učenje. Složeniji modeli se mogu izraditi korišćenjem polinomijalne ekspanzije u pripremi podataka. Neparametarski modeli ne podrazumevaju unapred poznatu funkciju mapiranja između ulaznih i izlaznih promenljivih. Oni su dosta fleksibilniji, ali su skloniji pojavi overfittinga, zahtevaju veću količinu podataka i sporije se treniraju.

Uzimajući u obzir način na koji najčešće korišćeni algoritmi rade, oni se mogu podeliti na sledeće grupe:
- Regresioni
- Algoritmi zasnovani na instancama
- Algoritmi zasnovani na stablima odlučivanja
- Algoritmi zasnovani na kernelima
- Bajezijan metodi
- Kombinovani (ensemble) metodi

Za svaki od ovih algoritama postoji odgovarajuća implementacija u Python-u. Ona podrazumeva i podešavanje hiper-parametara, karakterističnih za svaki od ovih algoritama.

## Osnovni postupak kreiranja modela mašinskog učenja primenom različitih algoritama

Za kreiranje modela mašinskog učenja primenom različitih algoritama, koriste se funkcije iz Python paketa sklearn, odnosno podpaketa koji obuhvataju funkcije relevantne za pojedinačne algoritme. Dole navedeni kod uvozi funkcije koje se koriste za inicijalizaciju svih algoritama korišćenih u primeru demonstracije osnovnog postupka kreiranja modela mašinskog učenja, njihovih podešavanja i analiza performansi, kao i funkcije relevantne za pripremu setova za učenje i testiranje.

In [1]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

Postupak se odnosi na definisanje različitih parametara i aktivnosti nad modelom koji se inicijalno kreira kao instanca određenog algoritma. Postupak se razlikuje u zavisnosti od toga da li se testiranje vrši metodom unakrsne validacije ili podelom na set za učenje i testiranje.

Za slučaj podele na set za učenje i testiranje, postupak obuhvata sledeće faze:
- Podela skupa na podskupove za učenje i testiranje, npr. X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
- Inicijalizacija modela, korišćenjem odgovarajućih funkcija sklearn paketa, npr. model=LogisticRegression()
- Učenje modela skupom za učenje, npr. model.fit(X_train, Y_train)
- Predviđanje izlaznih veličina, npr. predictions = model.predict(X_test)
- Prikaz veličina kvaliteta modela, npr. print(metrics.accuracy_score(Y_test, predictions))

Za slučaj unakrsne validacije, postupak obuhvata sledeće faze:
- Inicijalizacija modela, korišćenjem odgovarajućih funkcija sklearn paketa, npr. model=LogisticRegression()
- Učenje modela, npr. model.fit(X, Y)
- Kreiranje iteratora za unakrsno testiranje, npr. kf = KFold(n_splits=10, random_state=7)
- Testiranje i prikaz veličina kvaliteta modela, npr. results = cross_val_score(lr, X, Y, cv=kf, scoring=scoring)


## Dataset za demonstraciju i podela na set za učenje i testiranje

Za demonstraciju se koristi skup podataka za predviđanje dijabetesa. Skup je prethodno pripremljen. Izvršene su sledeće aktivnosti pripreme podataka:
- Tretman nedostajućih vrednosti
- Tretman outlier-a
- Transformacija zakošenih veličina
- Skaliranje: Normalizacija i standardizacija
- Klasterovanje veličine godina starosti i njeno kodiranje
- Kodiranje veličine class

In [2]:
import pandas as pd
df=pd.read_csv('kurs_diabetes_prep.csv')
df.head()

,preg,plas,pres,skin,mass,pedi,ageclass_-25,ageclass_25-35,ageclass_35-45,ageclass_45-55,ageclass_55-65,ageclass_65+,class
0,0.632412,0.670968,0.008196,0.778478,0.204779,-0.466809,0,0,0,1,0,0,1
1,0.000000,0.264516,-0.479728,0.048715,-0.867922,-1.046969,0,1,0,0,0,0,0
2,0.733952,0.896774,-0.642369,-0.072913,-1.373624,-0.397497,0,1,0,0,0,0,1
3,0.000000,0.290323,-0.479728,-0.681049,-0.638058,-1.789761,1,0,0,0,0,0,0
4,0.489301,0.600000,-2.594064,0.778478,1.660587,0.827678,0,1,0,0,0,0,1


Najpre se vrednosti ulaznih i izlaznih podataka transformišu u numpy nizove, a onda se pozivanjem funkcije train_test_split vrši kreiranje skupova ulaznih i izlaznih veličina za učenje i testiranje. Navedeno je da se 30% podataka (test_size=0.3), koristi za testiranje, a 70% za učenje.

In [3]:
X=df.values[:,:12]
Y=df.values[:,12]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=7)

## Kreiranje modela zasnovanog na logističkoj regresiji i njegovo učenje

Najpre, radi demonstracije postupka, kreiraće se model zasnovan na logističkoj regresiji, izvršiće se njegovo učenje i utvrđivanje tačnosti predviđanja. Prilikom inicijalizacije modela, kod dole definiše i vrednosti nekih hiperparametara. U ovom slučaju, definisan je algoritam koji se koristi za optimizaciju (liblinear) i regularizacioni parametar (podrazumevana vrednost je 1.0, manja vrednost znači snažniju regularizaciju).

In [4]:
model=LogisticRegression(solver='liblinear',
                        C=0.8)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
print(metrics.accuracy_score(Y_test, predictions))

0.7347826086956522


Dole je prikazan kod koji vrši kreiranje istovetnog modela, ali se testiranje vrši metodom unakrsne validacije. Ovaj metod daje relevantniji i pouzdaniji rezultat, nezavisno od toga da li je dobijena vrednost tačnosti veća ili manja od one dobijene primenom train-test split metode.

In [5]:
model=LogisticRegression(solver='liblinear',
                        C=0.8)
kf = KFold(n_splits=10, random_state=7)
model.fit(X, Y)
results = cross_val_score(model, X, Y, cv=kf, scoring='accuracy')
score= model.score(X_test, Y_test)
print(results.mean())

0.7724025974025974


### Ocenjivanje performansi modela koji se koristi za rešavanje problema klasifikacije

Osnovna performansa modela koji se koristi za rešavanje problema klasifikacije je tačnost predviđanja. Ona predstavlja odnos tačnih predviđanja i ukupnog broja predviđanja. Ona se smatra samo početnim indikatorom za analizu performansi i nije pouzdana jer zavisi od raspodele vrednosti izlazne promenljive. Na primer, primenom train-test split metode testiranja, dobijena je tačnost of 0.739. Iako se radi o binarnoj klasifikaciji, referentna tačnost nije 50% već 65%, zbog neravnomerne zastupljenosti pacijenata koji imaju dijabetes i onih koji ga nemaju.

Tačnost modela koja se dobija korišćenjem unakrsne validacije je prosečna vrednost tačnosti koja se utvrđuje u svakom od prolaza (k).

In [6]:
print(metrics.accuracy_score(Y_test, predictions))

0.7347826086956522


Logaritamski gubitak je mera pouzdanosti predviđanja. Naime, tačnost uzima u obzir samo predviđene vrednosti, odnosno u ovom slučaju 0 ili 1. Logaritamski gubitak uzima u obzir vrednosti proračunate vrednosti, koje su tek naknadno svedene na 0 ili 1, na primer, primenom neke aktivacione funkcije (npr. sigmoid u logističkoj regresiji). Drugim rečima, ova mera se koristi da se utvrdi koliko je model "siguran" u svoje predviđanje.

In [7]:
print(metrics.log_loss(Y_test, predictions))

9.160350228003153


Confusion matrica se koristi radi utvrđivanja sposobnosti razlikovanja klasa od strane modela. Uvidom u confusion matricu, moguće je utvrditi broj tačnih (brojevi po dijagonali matrice) i netačnih predviđanja pojedinačnih klasa. Uvidom u matricu dole, moguće je zaključiti:

- Model je 47 puta tačno predvideo da pacijent IMA dijabetes (True Positive - TP)
- Model je 123 puta tačno predvideo da pacijent NEMA dijabetes (True Negative - TN)
- Model je 20 puta predvideo da pacijent ima dijabetes, ali to nije bilo tačno (False Positive - FP)
- Model je 40 puta predvideo da pacijent nema dijabetes, ali to nije bilo tačno (False Negative - FN)

Confusion matricu je nemoguće formirati u slučaju kada se koristi unakrsna validacija, jer ona podrazumeva višestruko testiranje.

In [8]:
print(confusion_matrix(Y_test, predictions, labels=[1,0]))

[[ 45  42]
 [ 19 124]]


Mere performansi koje uzimaju u obzir distribuciju tačnih predviđanja po klasama izlazne veličine su preciznost (precision) i osetljivost (recall). Obe ove mere se određuju po svakoj vrednosti izlazne veličine i mogu se utvrditi štampanjem tzv. klasifikacionog izveštaja, korišćenjem odgovarajuće metode, kao što je prikazano dole.

Preciznost ukazuje na to koji deo predviđanja jedne klase je ispao tačan. Odnosno, precision=TP/(TP+FP). Na primeru predviđanja dijabetesa, to se može tumačiti na sledeći način:
- Od svih predviđenih dijagnoza dijabetesa, koje su se ispostavile kao tačne? Na našem primeru, 47/(47+20)=0.70. Odnosno, kada model predviđa da pacijent ima dijabetesa, 70% puta je u pravu.
- Od svih predviđenih dijagnoza zdravih pacijenata, koje su se ispostavile kao tačne? Na našem primeru, 123/(123+40)=0.754. Odnosno, kada model predviđa da pacijent nema dijabetesa, 75.4% puta je u pravu.

Osetljivost se definiše u suprotnom kontekstu. Ona određuje odnos tačnih predviđanja jedne klase izlazne veličine i ukupan broj vrednosti te klase. Odnosno, recall=TP/(TP+FN). Na primeru predviđanja dijabetesa, to se može tumačiti na sledeći način:
- Od svih pacijenata koji imaju dijabetes, za koji deo se postavila dijagnoza dijabetesa? Na našem primeru, 47/(47+20)=0.70. Odnosno, model 70% puta tačno predviđa dijagnozu dijabetesa.
- Od svih pacijenata koji nemaju dijabetes, za koji deo se postavila takva dijagnoza? Na našem primeru, 123/(123+20)=0.86. Odnosno, model 86% puta tačno predviđa dijagnozu zdravog pacijenta.

F1 skor je mera koja kombinuje preciznost i osetljivost: f1=2*Precision*Recall/(Precision+Recall).

In [9]:
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.75      0.87      0.80       143
         1.0       0.70      0.52      0.60        87

   micro avg       0.73      0.73      0.73       230
   macro avg       0.73      0.69      0.70       230
weighted avg       0.73      0.73      0.72       230



Jedna od mera performansi koja se koristi za ocenu sposobnosti modela da razlikuje klase izlazne veličine je površina ispod ROC krive (Area Under ROC Curve, AUROC). Ona se može koristiti samo za binarnu klasifikaciju i karakterišu je vrednosti u intervalu (0.5,1). AUROC=0.5 znači da model nije sposoban da razlikuje klase. AUROC=1 znači da model potpuno razlikuje klase, sva njegova predviđanja su tačna.

In [10]:
print(metrics.roc_auc_score(Y_test, predictions))

0.6921871232216059


### Optimizacija hiper-parametara modela

Svaki model, odnosno svaki algoritam karakteriše niz hiper-parametara čijim se podešavanjem (odnosno, izborom različitih kombinacija vrednosti) mogu unaprediti performanse modela. Kao postupak optimizacije se često koristi tzv. grid search. Ovaj metod podrazumeva korišćenje tzv. brutalne sile (brute force) u pronalažanju optimalne kombinacije hiper-parametara, na osnovu zadatih intervala vrednosti svakog od njih i mere na osnovu koje se vrši optimizacija.

Kod dole vrši optimizaciju vrednosti tri hiper-parametra i to uzimajući u obzir samo kombinacije vrednosti zadatih u asocijativnom nizu parameters. Očigledno, model je optimizovan, jer je vrednost ROCAUC skora veća od one koja je dobijena sa gore korišćenom kombinacijom parametara.

In [11]:
model = LogisticRegression()
parameters = {'C':[0.001,0.1,1,4,10,15,100],
              'penalty': ['l1','l2'],
              'solver': ['liblinear']
            }
grid = GridSearchCV(model, parameters, scoring=make_scorer(roc_auc_score))
grid = grid.fit(X_train, Y_train)
print("Parameters :", grid.best_params_)
print("ROCAUC :", grid.best_score_)

Parameters : {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
ROCAUC : 0.7345402796426131


## Kreiranje modela sa često korišćenim algoritmima i optimizacija hiper-parametara

Radi upoređenja performansi modela izrađenih korišćenjem različitih algoritama, kreirana je funkcija testModel koja prihvata inicijalizovan model kao ulaznu promenljivu, trenira taj model sa podacima iz seta za učenje, vrši predviđanje izlazne veličine na osnovu ulaznih podataka iz seta za testiranje i štampa dve osnovne mere performansi: tačnost i ROCAUC skor (mera sposobnosti razdvajanja klasa). Samo random_state hiper parametar je postavljen, zbog mogućnosti kasnijeg upoređivanja mera performansi.

In [12]:
def testModel(m,model):
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)
    print(m)
    print('-------------------------')
    print("Accuracy:", metrics.accuracy_score(Y_test, predictions))
    print("ROCAUC score:", metrics.roc_auc_score(Y_test, predictions))
    print('-------------------------')
    
testModel('LR',LogisticRegression(random_state=7))
testModel('DT',DecisionTreeClassifier(random_state=7))
testModel('KNN',KNeighborsClassifier())
testModel('SVC',SVC(random_state=7))
testModel('RF',RandomForestClassifier(random_state=7))

LR
-------------------------
Accuracy: 0.7391304347826086
ROCAUC score: 0.6979342496583876
-------------------------
DT
-------------------------
Accuracy: 0.7086956521739131
ROCAUC score: 0.6937143316453661
-------------------------
KNN
-------------------------
Accuracy: 0.6695652173913044
ROCAUC score: 0.6284864560726628
-------------------------
SVC
-------------------------
Accuracy: 0.7043478260869566
ROCAUC score: 0.6452053693433004
-------------------------
RF
-------------------------
Accuracy: 0.7086956521739131
ROCAUC score: 0.6599549875411944
-------------------------


Unapređena verzija koda je prikazana dole. Nova funkcija, pored prikaza osnovne metrike, vrši i optimizaciju hiper-parametara i to na osnovu grida - niza parametara koji se dostavlja kao argument.

In [13]:
def testModel(m, model, parameters):
    grid = GridSearchCV(model, parameters, scoring=make_scorer(roc_auc_score))
    grid = grid.fit(X_train, Y_train)
    print(m)
    print('-------------------------')
    print("Parameters :", grid.best_params_)
    print("ROCAUC Score :", grid.best_score_)
    print('-------------------------')
    
testModel('LR',LogisticRegression(random_state=7),
          {'C':[0.001,0.1,1,4,10,15,100],
           'penalty': ['l1','l2'],
           'solver': ['liblinear']})
testModel('DT',DecisionTreeClassifier(random_state=7),
          {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [9,10,11,12,13,14], 
          'min_samples_leaf':[3,4,5,6,7]})
testModel('KNN',KNeighborsClassifier(),
          {'n_neighbors':[8,9,10],
          'leaf_size':[1,2,3],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute'],
          'n_jobs':[-1]})
testModel('SVC',SVC(random_state=7),
          {'C': [7,8,9,10,11], 
          'kernel': ['linear','poly','rbf','sigmoid'],
          'gamma':['auto'],
          'degree': [2,3]})
testModel('RF',RandomForestClassifier(random_state=7),
          {'n_estimators': [2,4,6],
           'max_features': ['log2', 'sqrt','auto'],
           'criterion': ['entropy', 'gini'],
           'max_depth': [2,3,5],
           'min_samples_split': [2,3],
           'min_samples_leaf': [2,3]})

LR
-------------------------
Parameters : {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
ROCAUC Score : 0.7345402796426131
-------------------------


F:\Programs\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


DT
-------------------------
Parameters : {'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 13}
ROCAUC Score : 0.685371833936393
-------------------------
KNN
-------------------------
Parameters : {'algorithm': 'auto', 'leaf_size': 1, 'n_jobs': -1, 'n_neighbors': 9, 'weights': 'uniform'}
ROCAUC Score : 0.6248568388823187
-------------------------
SVC
-------------------------
Parameters : {'C': 11, 'degree': 2, 'gamma': 'auto', 'kernel': 'linear'}
ROCAUC Score : 0.7275019377394836
-------------------------
RF
-------------------------
Parameters : {'criterion': 'entropy', 'max_depth': 3, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 4}
ROCAUC Score : 0.7147269245411615
-------------------------


F:\Programs\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Značajna poboljšanja su vidljiva kod algoritama SVC i RandomForest, dok je kod stabla odlučivanja i KNN čak došlo do degradacije ROCAUC skora, što znači da podrazumevane vrednosti hiper-parametara nisu bile uključene u grid.